#### <b>필요한 라이브러리 설치 및 불러오기</b>

* <b>Transformer</b>: 트랜스포머 아키텍처 라이브러리
  * 한국어 모델을 포함해 다양한 큰 규모의(large-scale) 트랜스포머 모델(BERT, ELECTRA 등) 을 제공한다.
* <b>Soynlp</b>: 한국어를 위한 토큰화 라이브러리
  * 직접 사전(dictionary)를 직접 정의하지 않고도, <b>한국어</b>가 가지는 특수한 단어를 포함하여 토큰으로 만들 수 있도록 도와준다.

In [1]:
!pip install transformers
!pip install soynlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 8.4 MB/s eta 0:00:00


In [2]:
# PyTorch 라이브러리 불러오기
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# 사전 학습된 트랜스포머(Transformer) 모델 사용하기
from transformers import (
    ElectraPreTrainedModel,
    ElectraModel,
    ElectraConfig,
    ElectraTokenizer,
    BertPreTrainedModel,
    BertModel,
    BertConfig,
    BertTokenizer
)

# 트랜스포머(Transformer) 학습을 위한 라이브러리 불러오기
from transformers import AdamW, get_linear_schedule_with_warmup

# 한국어 모델 학습을 위한 정규화 라이브러리 사용
from soynlp.normalizer import emoticon_normalize, repeat_normalize

# 기타 라이브러리 불러오기
import os
import re
import copy
import json
import logging
import random
import numpy as np

# 모델 학습 및 학습된 모델 평가를 위한 라이브러리
from sklearn.metrics import f1_score
from tqdm import tqdm, trange

# 하이퍼 파라미터 정의 목적
from argparse import Namespace
from types import SimpleNamespace

#### <b>헤드(Head) 모델 정의하기</b>

* 실제로 내가 원하는 기능을 수행하는 모델을 정의한다.
* <b>헤드(head)</b>: 잘 학습된 모델 뒤에 붙여서 특정한 기능을 위한 작은 크기의 네트워크 모델
  * BERT 혹은 ELECTRA를 통해 추출된 특징(feature)를 이용해 실제로 분류 기능을 수행할 수 있다.

In [3]:
class BiasClassificationHead(nn.Module):
    """Head for Bias Classification"""

    def __init__(self, config, num_bias_labels):
        super().__init__()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_bias_labels)

    def forward(self, x):
        x = self.dropout(x)
        x = self.classifier(x)
        return x


class HateClassificationHead(nn.Module):
    """Head for Hate Classification"""

    def __init__(self, config, num_hate_labels):
        super().__init__()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_hate_labels)

    def forward(self, x):
        x = self.dropout(x)
        x = self.classifier(x)
        return x

#### <b>특징 추출기(Feature Extractor) 모델 정의하기</b>

* 하나의 텍스트(text) 입력이 주어졌을 때, 해당 텍스트에 대하여 특징(feature)을 추출한다.
  * 대표적인 Large Language 모델인 BERT와 ELECTRA를 불러와 사용할 수 있다.

In [4]:
class ElectraForBiasClassification(ElectraPreTrainedModel):
    def __init__(self,
                 config: ElectraConfig,
                 args: Namespace,
                 bias_label_lst=None,
                 hate_label_lst=None):
        super().__init__(config)
        self.args = args
        self.num_bias_labels = len(bias_label_lst) if bias_label_lst is not None else 0
        self.num_hate_labels = len(hate_label_lst) if hate_label_lst is not None else 0

        self.electra = ElectraModel(config) # 특징 추출기

        # 편견(bias)와 혐오(hate) 분류 모델을 동시에 학습한다. (multi-task 상황)
        self.bias_classifier = BiasClassificationHead(config, self.num_bias_labels)
        self.hate_classifier = HateClassificationHead(config, self.num_hate_labels)

        # 여러 개의 클래스 중에서 하나로 예측하는 multi-class classification 상황 가정
        self.loss_fct = nn.CrossEntropyLoss()

        self.init_weights()

    # 실제로 모델을 호출했을 때 사용하는 함수
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        bias_labels=None,
        hate_labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        discriminator_hidden_states = self.electra(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states
        )

        pooled_output = discriminator_hidden_states[0][:, 0]

        bias_logits = self.bias_classifier(pooled_output)
        hate_logits = self.hate_classifier(pooled_output)

        total_loss = 0

        # 편견(bias) 모델 사용하기
        if bias_labels is not None:
            bias_loss = self.loss_fct(bias_logits.view(-1, self.num_bias_labels), bias_labels.view(-1))
            total_loss += self.args.bias_loss_coef * bias_loss

        # 혐오(hate) 모델 사용하기
        if hate_labels is not None:
            hate_loss = self.loss_fct(hate_logits.view(-1, self.num_hate_labels), hate_labels.view(-1))
            total_loss += self.args.hate_loss_coef * hate_loss

        outputs = ((bias_logits, hate_logits),) + discriminator_hidden_states[1:]  # add hidden states and attention if they are here

        outputs = (total_loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


class BertForBiasClassification(BertPreTrainedModel):
    def __init__(self,
                 config: BertConfig,
                 args: Namespace,
                 bias_label_lst=None,
                 hate_label_lst=None):
        super().__init__(config)
        self.args = args
        self.num_bias_labels = len(bias_label_lst) if bias_label_lst is not None else 0
        self.num_hate_labels = len(hate_label_lst) if hate_label_lst is not None else 0

        self.bert = BertModel(config) # 특징 추출기

        # 편견(bias)와 혐오(hate) 분류 모델을 동시에 학습한다. (multi-task 상황)
        self.bias_classifier = BiasClassificationHead(config, self.num_bias_labels)
        self.hate_classifier = HateClassificationHead(config, self.num_hate_labels)

        # 여러 개의 클래스 중에서 하나로 예측하는 multi-class classification 상황 가정
        self.loss_fct = nn.CrossEntropyLoss()

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        bias_labels=None,
        hate_labels=None,
        output_attentions=None,
        output_hidden_states=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )

        pooled_output = outputs[1]

        bias_logits = self.bias_classifier(pooled_output)
        hate_logits = self.hate_classifier(pooled_output)

        total_loss = 0

        # 편견(bias) 모델 사용하기
        if bias_labels is not None:
            bias_loss = self.loss_fct(bias_logits.view(-1, self.num_bias_labels), bias_labels.view(-1))
            total_loss += self.args.bias_loss_coef * bias_loss

        # 혐오(hate) 모델 사용하기
        if hate_labels is not None:
            hate_loss = self.loss_fct(hate_logits.view(-1, self.num_hate_labels), hate_labels.view(-1))
            total_loss += self.args.hate_loss_coef * hate_loss

        outputs = ((bias_logits, hate_logits),) + outputs[2:]  # add hidden states and attention if they are here

        outputs = (total_loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

#### <b>트랜스포머 모델에 관한 기본적인 라이브러리 및 설정</b>

In [5]:
MODEL_CLASSES = {
    "koelectra-base": (ElectraConfig, ElectraForBiasClassification, ElectraTokenizer),
    "koelectra-small": (ElectraConfig, ElectraForBiasClassification, ElectraTokenizer),
    "koelectra-base-v2": (ElectraConfig, ElectraForBiasClassification, ElectraTokenizer),
    "koelectra-small-v2": (ElectraConfig, ElectraForBiasClassification, ElectraTokenizer),
    "kcbert-base": (BertConfig, BertForBiasClassification, BertTokenizer),
}


def load_tokenizer(args):
    return MODEL_CLASSES[args.model_type][2].from_pretrained(args.model_name_or_path)


def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if not args.no_cuda and torch.cuda.is_available():
        torch.cuda.manual_seed_all(args.seed)


def compute_metrics(pred_bias_labels, pred_hate_labels, gt_bias_labels, gt_hate_labels):
    bias_weighted_f1 = f1_score(gt_bias_labels, pred_bias_labels, average="weighted")
    hate_weighted_f1 = f1_score(gt_hate_labels, pred_hate_labels, average="weighted")

    bias_macro_f1 = f1_score(gt_bias_labels, pred_bias_labels, average="macro")
    hate_macro_f1 = f1_score(gt_hate_labels, pred_hate_labels, average="macro")

    mean_weighted_f1 = (bias_weighted_f1 + hate_weighted_f1) / 2
    return {
        "bias_weighted_f1": bias_weighted_f1,
        "hate_weighted_f1": hate_weighted_f1,
        "mean_weighted_f1": mean_weighted_f1,
        "bias_macro_f1": bias_macro_f1,
        "hate_macro_f1": hate_macro_f1
    }

#### <b>한국어 혐오 표현 분류 모델을 위한 전처리 부분</b>

* 내가 원하는 데이터 세트에 대하여, 데이터를 불러오는 함수를 작성할 필요가 있다.

In [6]:
# 제목(title)과 댓글(comment)을 전처리하는 함수
def preprocess(title: str, comment: str):
    # Erase redundant \" in the start & end of the title
    if title.startswith("\""):
        title = title[1:]
    if title.endswith("\""):
        title = title[:-1]

    # Change quotes
    title = title.replace("“", "\"").replace("”", "\"").replace("‘", "\'").replace("’", "\'")

    # Erase braces in title
    braces = r"\[(.*?)\]"
    braces2 = r"\{(.*?)\}"
    braces3 = r"\【(.*?)\】"
    braces4 = r"\<(.*?)\>"

    title = re.sub(braces, '', title)
    title = re.sub(braces2, '', title)
    title = re.sub(braces3, '', title)
    title = re.sub(braces4, '', title)

    # Normalize the comment
    comment = emoticon_normalize(comment, num_repeats=3)
    comment = repeat_normalize(comment, num_repeats=3)

    return title, comment


# 하나의 텍스트(text) 입력에 대한 클래스
class InputExample(object):
    """ A single training/test example for simple sequence classification. """

    def __init__(self,
                 guid,
                 text_a,
                 text_b,
                 bias_label,
                 hate_label):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.bias_label = bias_label
        self.hate_label = hate_label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


# 하나의 텍스트 입력의 특징(feature)에 대한 클래스
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 input_ids,
                 attention_mask,
                 token_type_ids,
                 bias_label=None,
                 hate_label=None):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids
        self.bias_label = bias_label
        self.hate_label = hate_label

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"


# 한국어 혐오 표현 전처리 기능 클래스
class KoreanHateSpeechProcessor(object):
    """Processor for the Korean Hate Speech data set """

    def __init__(self, args):
        self.args = args

    @classmethod
    def get_labels(cls):
        bias_label_lst = ['none', 'gender', 'others']
        hate_label_lst = ['none', 'hate', 'offensive']
        return bias_label_lst, hate_label_lst

    @classmethod
    def _read_file(cls, input_file):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            lines = []
            for line in f:
                lines.append(line.strip())
            return lines

    def _create_examples(self, lines, set_type):
        """Creates examples for the train, dev and test sets."""
        examples = []
        for (i, line) in enumerate(lines[1:]):  # Except the header
            line = line.split('\t')
            guid = "%s-%s" % (set_type, i)
            title = line[0]
            comment = line[1]
            title, comment = preprocess(title, comment)

            bias_label = None
            hate_label = None
            if set_type != "test":
                bias_label = line[2]
                hate_label = line[3]
            if i % 1000 == 0:
                print([title, comment])

            examples.append(InputExample(guid=guid,
                                         text_a=comment,
                                         text_b=title,
                                         bias_label=bias_label,
                                         hate_label=hate_label))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, dev, test
        """
        file_to_read = None
        if mode == 'train':
            file_to_read = self.args.train_file
        elif mode == 'dev':
            file_to_read = self.args.dev_file
        elif mode == 'test':
            file_to_read = self.args.test_file

        print("LOOKING AT {}".format(os.path.join(self.args.data_dir, file_to_read)))
        return self._create_examples(self._read_file(os.path.join(self.args.data_dir, file_to_read)), mode)


# 실질적으로 텍스트 입력을 트랜스포머 모델에 넣어 특징(feature)을 계산하는 함수
def convert_examples_to_features(
        examples,
        tokenizer,
        max_length,
):
    bias_label_list, hate_label_list = KoreanHateSpeechProcessor.get_labels()

    bias_label_map = {label: i for i, label in enumerate(bias_label_list)}
    hate_label_map = {label: i for i, label in enumerate(hate_label_list)}

    def label_from_example(example):
        bias_label_id = -1
        hate_label_id = -1
        if example.bias_label is not None:
            bias_label_id = bias_label_map[example.bias_label]
        if example.hate_label is not None:
            hate_label_id = hate_label_map[example.hate_label]
        return bias_label_id, hate_label_id

    labels = [label_from_example(example) for example in examples]

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples], max_length=max_length, pad_to_max_length=True
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        if "token_type_ids" not in inputs:
            inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  # For xlm-roberta, distilkobert

        feature = InputFeatures(**inputs, bias_label=labels[i][0], hate_label=labels[i][1])
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        print("*** Example ***")
        print("guid: {}".format(example.guid))
        print("input_ids: {}".format(" ".join([str(x) for x in features[i].input_ids])))
        print("attention_mask: {}".format(" ".join([str(x) for x in features[i].attention_mask])))
        print("token_type_ids: {}".format(" ".join([str(x) for x in features[i].token_type_ids])))
        print("bias_label: {}".format(features[i].bias_label))
        print("hate_label: {}".format(features[i].hate_label))

    return features


def load_examples(args, tokenizer, mode):
    processor = KoreanHateSpeechProcessor(args)

    print("Creating features from dataset file at %s", args.data_dir)
    if mode == "train":
        examples = processor.get_examples("train")
    elif mode == "dev":
        examples = processor.get_examples("dev")
    elif mode == "test":
        examples = processor.get_examples("test")
    else:
        raise Exception("For mode, Only train, dev, test is available")

    features = convert_examples_to_features(
        examples,
        tokenizer,
        args.max_seq_len
    )

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_bias_labels = torch.tensor([f.bias_label for f in features], dtype=torch.long)
    all_hate_labels = torch.tensor([f.hate_label for f in features], dtype=torch.long)

    dataset = TensorDataset(all_input_ids,
                            all_attention_mask,
                            all_token_type_ids,
                            all_bias_labels,
                            all_hate_labels)
    return dataset

#### <b>모델 학습을 위한 클래스 정의</b>

* 실질적으로 모델을 학습할 수 있도록 해준다.

In [7]:
class Trainer(object):
    def __init__(self, args, tokenizer, train_dataset=None, dev_dataset=None, test_dataset=None):
        self.args = args
        self.tokenizer = tokenizer
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        self.test_dataset = test_dataset

        self.bias_label_lst, self.hate_label_lst = KoreanHateSpeechProcessor.get_labels()

        self.config_class, self.model_class, _ = MODEL_CLASSES[args.model_type]

        self.config = self.config_class.from_pretrained(args.model_name_or_path,
                                                        finetuning_task=args.task)
        self.model = self.model_class.from_pretrained(args.model_name_or_path,
                                                      config=self.config,
                                                      args=args,
                                                      bias_label_lst=self.bias_label_lst,
                                                      hate_label_lst=self.hate_label_lst)

        # GPU or CPU
        self.device = "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
        self.model.to(self.device)

    def train(self):
        train_sampler = RandomSampler(self.train_dataset)
        train_dataloader = DataLoader(self.train_dataset, sampler=train_sampler, batch_size=self.args.train_batch_size)

        if self.args.max_steps > 0:
            t_total = self.args.max_steps
            self.args.num_train_epochs = self.args.max_steps // (len(train_dataloader) // self.args.gradient_accumulation_steps) + 1
        else:
            t_total = len(train_dataloader) // self.args.gradient_accumulation_steps * self.args.num_train_epochs

        # Prepare optimizer and schedule (linear warmup and decay)
        no_decay = ['bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in self.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'weight_decay': self.args.weight_decay},
            {'params': [p for n, p in self.model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.args.learning_rate, eps=self.args.adam_epsilon)
        scheduler = get_linear_schedule_with_warmup(optimizer,
                                                    num_warmup_steps=int(t_total * self.args.warmup_proportion),
                                                    num_training_steps=t_total)

        # Train!
        print("***** Running training *****")
        print("  Num examples = %d", len(self.train_dataset))
        print("  Num Epochs = %d", self.args.num_train_epochs)
        print("  Total train batch size = %d", self.args.train_batch_size)
        print("  Gradient Accumulation steps = %d", self.args.gradient_accumulation_steps)
        print("  Total optimization steps = %d", t_total)
        print("  Logging steps = %d", self.args.logging_steps)

        global_step = 0
        tr_loss = 0.0
        best_mean_weighted_f1 = 0.0
        self.model.zero_grad()

        train_iterator = trange(int(self.args.num_train_epochs), desc="Epoch")

        for _ in train_iterator:
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            for step, batch in enumerate(epoch_iterator):
                self.model.train()
                batch = tuple(t.to(self.device) for t in batch)  # GPU or CPU
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'bias_labels': batch[3],
                          'hate_labels': batch[4]}
                if self.args.model_type != 'distilkobert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                loss = outputs[0]

                if self.args.gradient_accumulation_steps > 1:
                    loss = loss / self.args.gradient_accumulation_steps

                loss.backward()

                tr_loss += loss.item()
                if (step + 1) % self.args.gradient_accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.args.max_grad_norm)

                    optimizer.step()
                    scheduler.step()  # Update learning rate schedule
                    self.model.zero_grad()
                    global_step += 1

                    if self.args.logging_steps > 0 and global_step % self.args.logging_steps == 0:
                        results = self.evaluate("dev")
                        if results["mean_weighted_f1"] > best_mean_weighted_f1:  # Save best result based on mean f1 score
                            best_mean_weighted_f1 = results["mean_weighted_f1"]
                            self.save_model()

                if 0 < self.args.max_steps < global_step:
                    epoch_iterator.close()
                    break

            if 0 < self.args.max_steps < global_step:
                train_iterator.close()
                break

        return global_step, tr_loss / global_step

    def evaluate(self, mode):
        if mode == 'test':
            dataset = self.test_dataset
        elif mode == 'dev':
            dataset = self.dev_dataset
        else:
            raise Exception("Only dev and test dataset available")

        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        print("***** Running evaluation on %s dataset *****", mode)
        print("  Num examples = %d", len(dataset))
        print("  Batch size = %d", self.args.eval_batch_size)
        eval_loss = 0.0
        nb_eval_steps = 0

        bias_preds = None
        bias_out_label_ids = None
        hate_preds = None
        hate_out_label_ids = None

        self.model.eval()

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'bias_labels': batch[3],
                          'hate_labels': batch[4]}
                if self.args.model_type != 'distilkobert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                tmp_eval_loss, (bias_logits, hate_logits) = outputs[:2]

                eval_loss += tmp_eval_loss.mean().item()
            nb_eval_steps += 1

            # Bias
            if bias_preds is None:
                bias_preds = bias_logits.detach().cpu().numpy()
                bias_out_label_ids = inputs['bias_labels'].detach().cpu().numpy()
            else:
                bias_preds = np.append(bias_preds, bias_logits.detach().cpu().numpy(), axis=0)
                bias_out_label_ids = np.append(
                    bias_out_label_ids, inputs['bias_labels'].detach().cpu().numpy(), axis=0)

            # Hate
            if hate_preds is None:
                hate_preds = hate_logits.detach().cpu().numpy()
                hate_out_label_ids = inputs['hate_labels'].detach().cpu().numpy()
            else:
                hate_preds = np.append(hate_preds, hate_logits.detach().cpu().numpy(), axis=0)
                hate_out_label_ids = np.append(
                    hate_out_label_ids, inputs['hate_labels'].detach().cpu().numpy(), axis=0)

        eval_loss = eval_loss / nb_eval_steps
        results = {
            "loss": eval_loss
        }

        bias_preds = np.argmax(bias_preds, axis=1)
        hate_preds = np.argmax(hate_preds, axis=1)
        result = compute_metrics(bias_preds, hate_preds, bias_out_label_ids, hate_out_label_ids)
        results.update(result)

        print("***** Eval results *****")
        for key in sorted(results.keys()):
            print("  %s = %s", key, str(results[key]))

        return results

    def predict(self):
        # Predict the test dataset which doesn't have label
        dataset = self.test_dataset
        eval_sampler = SequentialSampler(dataset)
        eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=self.args.eval_batch_size)

        # Eval!
        print("***** Running prediction on test dataset *****")
        print("  Num examples = %d", len(dataset))
        print("  Batch size = %d", self.args.eval_batch_size)

        bias_preds = None
        hate_preds = None

        self.model.eval()

        for batch in tqdm(eval_dataloader, desc="Evaluating"):
            batch = tuple(t.to(self.device) for t in batch)
            with torch.no_grad():
                inputs = {'input_ids': batch[0],
                          'attention_mask': batch[1],
                          'bias_labels': None,
                          'hate_labels': None}
                if self.args.model_type != 'distilkobert':
                    inputs['token_type_ids'] = batch[2]
                outputs = self.model(**inputs)
                _, (bias_logits, hate_logits) = outputs[:2]

            # Bias
            if bias_preds is None:
                bias_preds = bias_logits.detach().cpu().numpy()
            else:
                bias_preds = np.append(bias_preds, bias_logits.detach().cpu().numpy(), axis=0)

            # Hate
            if hate_preds is None:
                hate_preds = hate_logits.detach().cpu().numpy()
            else:
                hate_preds = np.append(hate_preds, hate_logits.detach().cpu().numpy(), axis=0)

        bias_preds = np.argmax(bias_preds, axis=1).tolist()
        hate_preds = np.argmax(hate_preds, axis=1).tolist()

        # Write the result
        print("Writing Prediction to {}...".format(self.args.prediction_file))
        if not os.path.exists(self.args.pred_dir):
            os.makedirs(self.args.pred_dir)
        with open(os.path.join(self.args.pred_dir, self.args.prediction_file), "w", encoding="utf-8") as f:
            f.write("bias,hate\n")
            for bias_idx, hate_idx in zip(bias_preds, hate_preds):
                f.write("{},{}\n".format(self.bias_label_lst[bias_idx], self.hate_label_lst[hate_idx]))

    def save_model(self):
        # Save model checkpoint (Overwrite)
        if not os.path.exists(self.args.model_dir):
            os.makedirs(self.args.model_dir)
        model_to_save = self.model.module if hasattr(self.model, 'module') else self.model
        model_to_save.save_pretrained(self.args.model_dir)
        self.tokenizer.save_pretrained(self.args.model_dir)

        # Save training arguments together with the trained model
        torch.save(self.args, os.path.join(self.args.model_dir, 'training_args.bin'))
        print("Saving model checkpoint to %s", self.args.model_dir)

    def load_model(self):
        # Check whether model exists
        if not os.path.exists(self.args.model_dir):
            raise Exception("Model doesn't exists! Train first!")

        self.config = self.config_class.from_pretrained(self.args.model_dir)
        self.model = self.model_class.from_pretrained(self.args.model_dir,
                                                      config=self.config,
                                                      args=self.args,
                                                      bias_label_lst=self.bias_label_lst,
                                                      hate_label_lst=self.hate_label_lst)

        self.model.to(self.device)
        print("***** Model Loaded *****")

#### <b>한국어 혐오 데이터 세트 불러오기</b>

In [8]:
!git clone https://github.com/monologg/korean-hate-speech-koelectra
%cd korean-hate-speech-koelectra

Cloning into 'korean-hate-speech-koelectra'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 44 (delta 18), reused 30 (delta 10), pack-reused 0
Unpacking objects: 100% (44/44), 769.20 KiB | 4.72 MiB/s, done.
/content/korean-hate-speech-koelectra


#### <b>하이퍼 파라미터 및 인자 값 설정하기</b>

In [9]:
args = SimpleNamespace()

args.task = "korean-hate-speech" # The name of the task to train

args.model_dir = "./model" # Path to save, load model
args.data_dir = "./data" # The input data dir
args.pred_dir = "./preds" # Directory that saves prediction files

args.train_file = "train.txt" # Train file
args.dev_file = "validate.txt" # Dev file
args.test_file = "test.txt" # Test file
args.prediction_file = "prediction.csv" # Output file for prediction

args.model_type = "koelectra-base-v2" # Model type selected
args.model_name_or_path = "monologg/koelectra-base-v2-discriminator" # Model name or path

args.seed = 42 # random seed for initialization
args.train_batch_size = 16 # Batch size for training.
args.eval_batch_size = 32 # Batch size for evaluation.
args.max_seq_len = 100 # The maximum total input sequence length after tokenization.
args.learning_rate = 5e-5 # The initial learning rate for Adam.
args.num_train_epochs = 10.0 # Total number of training epochs to perform.
args.weight_decay = 0.0 # Weight decay if we apply some.
args.gradient_accumulation_steps = 1 # Number of updates steps to accumulate before performing a backward/update pass.
args.adam_epsilon = 1e-8 # Epsilon for Adam optimizer.
args.max_grad_norm = 1.0 # Max gradient norm.
args.max_steps = -1 # If > 0: set total number of training steps to perform. Override num_train_epochs.
args.warmup_proportion = 0.1 # Warmup proportion for linear warmup

args.logging_steps = 200 # Log and save every X updates steps.

args.do_train = True # Whether to run training.
args.do_pred = True # Whether to run prediction on the test set.
args.no_cuda = False # Avoid using CUDA when available

args.bias_loss_coef = 0.5 # Coefficient for the bias loss.
args.hate_loss_coef = 1.0 # Coefficient for the hate loss.

#### <b>모델 학습을 위한 객체 초기화</b>

In [10]:
set_seed(args)

tokenizer = load_tokenizer(args)
train_dataset = load_examples(args, tokenizer, mode="train")
dev_dataset = load_examples(args, tokenizer, mode="dev")
test_dataset = load_examples(args, tokenizer, mode="test")
trainer = Trainer(args, tokenizer, train_dataset, dev_dataset, test_dataset)

Creating features from dataset file at %s ./data
LOOKING AT ./data/train.txt
['밤새 조문 행렬…故 전미선, 동료들이 그리워하는 따뜻한 배우 ', '(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네....']
['\'前 여자친구 피소\' 김정훈, 사흘 침묵→"원만한 해결 위해 노력" ', '근데 솔직히 김정훈 천재유전자 받고 싶어하는 여자들 많을텐데..']
[' 주진모 향한 조롱, 위기 극복 할까?', '댓글 성비랑 나이대보면 소름이 돋는다. 30~40대 아지매들 밥쳐먹고 할일없어 연예인한테 악플다는게 취미인가 보네.']
['최종훈, 집단 성폭행 의혹…""동석했지만 성관계 NO"" ', '범죄를 범죄인줄 모르고 ㅈㄹ는놈들이나 남자들 무데기로 있는 호텔방가서 ㅊ는 ㄴ나 에혀저런 ㄴ 만나결혼하면 인생 망친다']


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


["'황금빛' 신혜선, 천호진 위암 말기 사실 알았다..'충격' ", '아니ᆢᆢ지안이 황금빛내인생은 언제부터야ᆢᆢ']
['오또맘, \'장성규 팔로우\' 게시글 결국 사과..""경솔하게 행동했다"" ', '예쁘다. 이혼녀라도 환영한다']
[' 티아라, 前소속사와 맞선다..""상표출원 거절사유 제출""', '이제는 간~~다. 간~~~다 떠나는 이아픔. 왕따해서~~이미지추락한. ㅌ~~아라. 간~~~다. 간~~~다~~이젠. 홀로서기~~목매워 노래 부르네~~']
["'너노들' 연우진♥김세정, 달콤한 키스…김시후 죽음의 진실은(종합)", '진짜 연기못하는 애들만 모아뒀네 월.화 드라마가시청률 똥망이라 드라마 잠정 폐지까지 한다는데연기못하는 아이돌들은 왜자꾸 갔다가 쓰는건지 몰겠네쌍수해서 얼굴쳐다보는것도 졸부담이구 소름돋는 발연기에 그러니 시청률이 2%지']


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowin

*** Example ***
guid: train-0
input_ids: 2 36 401 1840 29995 29972 5676 39 44 9920 697 12741 22427 29951 192 28327 30317 29955 1840 30275 29959 30007 30058 30593 29950 30090 4592 765 4728 30160 30326 5 5 5 5 3 14340 9617 10377 1 7768 20158 30014 18 2617 3782 18853 29959 29950 3269 29957 851 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
bias_label: 2
hate_label: 1
*** Example ***
guid: train-1
input_ids: 2 5 5 5 5 131 30006 29972 12045 29953 237 30006 29972 107 5 5 5 5931 7518 29955 22804 14328 23495 5 5 5 46 2349

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Creating features from dataset file at %s ./data
LOOKING AT ./data/validate.txt
["'아스달 연대기' 뇌안탈 후계자 송중기, 이아르크에 숨어들었다 ", '송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.']
*** Example ***
guid: dev-0
input_ids: 2 18867 840 30315 29961 1500 29955 6075 5 506 30016 4505 29959 29955 300 30190 29948 5 3 40 6050 30124 2420 29960 40 1818 30043 30419 11561 18867 18 6 11840 30169 29951 1200 14961 30080 29948 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
bias_label: 0
hate_label: 0
*** Example ***
guid: dev-1
input_id

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


*** Example ***
guid: test-0
input_ids: 2 1 3580 71 29989 29973 29995 29950 1052 29992 204 29989 29962 300 30251 29948 1 512 29992 29961 446 30039 29956 124 29998 30365 31103 1 3 19742 446 30039 18 2427 2930 30171 30068 40 137 30539 29953 40 2368 147 30190 29948 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
token_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
bias_label: -1
hate_label: -1
*** Example ***
guid: test-1
input_ids: 2 932 29948 447 300 29948 260 1555 29959 11294 3 1 18 596 30036 30306 88 30027 30084 3915 9972 322 11711 5 5 41 41 1835 613 179

Some weights of the model checkpoint at monologg/koelectra-base-v2-discriminator were not used when initializing ElectraForBiasClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForBiasClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForBiasClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForBiasClassification were not initialized from the model checkpoint at monologg/koelectra-base-v2-discriminator and are newly initialized: ['hate_classifie

#### <b>모델 학습</b>

In [ ]:
if args.do_train:
    trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


***** Running training *****
  Num examples = %d 7896
  Num Epochs = %d 10.0
  Total train batch size = %d 16
  Gradient Accumulation steps = %d 1
  Total optimization steps = %d 4940.0
  Logging steps = %d 200


Iteration:  40%|████      | 199/494 [00:54<01:19,  3.73it/s]

***** Running evaluation on %s dataset ***** dev
  Num examples = %d 471
  Batch size = %d 32




Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Evaluating:   7%|▋         | 1/15 [00:00<00:02,  6.16it/s]

Evaluating:  13%|█▎        | 2/15 [00:00<00:02,  6.04it/s]

Evaluating:  20%|██        | 3/15 [00:00<00:02,  5.94it/s]

Evaluating:  27%|██▋       | 4/15 [00:00<00:01,  5.86it/s]

Evaluating:  33%|███▎      | 5/15 [00:00<00:01,  5.92it/s]

Evaluating:  40%|████      | 6/15 [00:01<00:01,  5.90it/s]

Evaluating:  47%|████▋     | 7/15 [00:01<00:01,  5.89it/s]

Evaluating:  53%|█████▎    | 8/15 [00:01<00:01,  5.89it/s]

Evaluating:  60%|██████    | 9/15 [00:01<00:01,  5.89it/s]

Evaluating:  67%|██████▋   | 10/15 [00:01<00:00,  5.86it/s]

Evaluating:  73%|███████▎  | 11/15 [00:01<00:00,  5.87it/s]

Evaluating:  80%|████████  | 12/15 [00:02<00:00,  5.88it/s]

Evaluating:  87%|████████▋ | 13/15 [00:02<00:00,  5.87it/s]

Evaluating:  93%|█████████▎| 14/15 [00:02<00:00,  5.87it/s]

Evaluating: 100%|██████████| 15/15 [00:02<00:00,  5.95it/s]


...


Iteration:  71%|███████▏  | 353/494 [01:42<00:39,  3.54it/s]

***** Running evaluation on %s dataset ***** dev
  Num examples = %d 471
  Batch size = %d 32




Evaluating:   0%|          | 0/15 [00:00<?, ?it/s]

Evaluating:   7%|▋         | 1/15 [00:00<00:02,  5.70it/s]

Evaluating:  13%|█▎        | 2/15 [00:00<00:02,  5.45it/s]

Evaluating:  20%|██        | 3/15 [00:00<00:02,  5.46it/s]

Evaluating:  27%|██▋       | 4/15 [00:00<00:02,  5.39it/s]

Evaluating:  33%|███▎      | 5/15 [00:00<00:01,  5.33it/s]

Evaluating:  40%|████      | 6/15 [00:01<00:01,  5.37it/s]

Evaluating:  47%|████▋     | 7/15 [00:01<00:01,  5.39it/s]

Evaluating:  53%|█████▎    | 8/15 [00:01<00:01,  5.37it/s]

Evaluating:  60%|██████    | 9/15 [00:01<00:01,  5.35it/s]

Evaluating:  67%|██████▋   | 10/15 [00:01<00:00,  5.35it/s]

Evaluating:  73%|███████▎  | 11/15 [00:02<00:00,  5.38it/s]

Evaluating:  80%|████████  | 12/15 [00:02<00:00,  5.41it/s]

Evaluating:  87%|████████▋ | 13/15 [00:02<00:00,  5.41it/s]

Evaluating:  93%|█████████▎| 14/15 [00:02<00:00,  5.35it/s]

Evaluating: 100%|██████████| 15/15 [00:02<00:00,  5.45it/s]

Iteration:  72%|███████▏  | 354/494 [01:

***** Eval results *****
  %s = %s bias_macro_f1 0.693354949679669
  %s = %s bias_weighted_f1 0.7961902000484149
  %s = %s hate_macro_f1 0.6800958198844821
  %s = %s hate_weighted_f1 0.6759342766510119
  %s = %s loss 2.6339526335398356
  %s = %s mean_weighted_f1 0.7360622383497134



Epoch: 100%|██████████| 10/10 [25:01<00:00, 150.14s/it]


#### <b>모델 평가</b>

In [ ]:
if args.do_pred:
    trainer.load_model()
    trainer.predict()

***** Model Loaded *****
***** Running prediction on test dataset *****
  Num examples = %d 974
  Batch size = %d 32


Evaluating: 100%|██████████| 31/31 [00:05<00:00,  5.53it/s]

Writing Prediction to prediction.csv...
